In [2]:
import torch
import torch.nn as nn
import timm

# Res2Net modelini yükleme
model = timm.create_model('res2net50_26w_4s', pretrained=True)

# Sınıf sayısına göre çıkış katmanını ayarlama (örneğin, CIFAR-10 için 10 sınıf var)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 10)  # CIFAR-10 veri kümesi için çıkış boyutunu 10 olarak ayarlayın

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


In [3]:
import torchvision.transforms as transforms
import torchvision.datasets as datasets

# Veri kümesini yükleme ve işleme
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)


100%|████████████████████████| 170498071/170498071 [00:41<00:00, 4062302.94it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [4]:
import torchvision.transforms as transforms
import torchvision.datasets as datasets

# Veri kümesini yükleme ve işleme
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)


Files already downloaded and verified
Files already downloaded and verified


In [5]:
import torch.optim as optim
from torch.utils.data import DataLoader

# DataLoader'ları oluşturma
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Loss ve optimizer tanımlama
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Modeli eğitme
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        accuracy = correct / total
        print(f'Epoch {epoch+1}/{num_epochs}, Test Accuracy: {accuracy}')


Epoch 1/5, Test Accuracy: 0.7772
Epoch 2/5, Test Accuracy: 0.8225
Epoch 3/5, Test Accuracy: 0.8554
Epoch 4/5, Test Accuracy: 0.8775
Epoch 5/5, Test Accuracy: 0.8701
